In [ ]:
#!pip3 install seaborn pandas matplotlib numpy    

every successive data of each device id were in an interval of approx 60 sec

In [ ]:
import csv 
import pandas as pd 
from datetime import date , time 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import warnings
from traceback import print_exc
import datetime
file_names = ["m1.csv" , "m2.csv" , "m3.csv" , "m4.csv"]
save_path = "combined_mahanagar_data.csv" 
rcp = "rush_clasified.csv" #rush_classified_path
dpd = "data_per_day.csv" #data_per_day
save_path_2 = "data_tracking.csv"
columns = ["distance" , "totalDistance" , "deviceId","fixTime","latitude","longitude","speed"]
total_bus , total_dates = [],[]
dates_given = ['2020-02-23', '2019-08-22', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-06', '2020-01-27', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20']
dict_dates = {
        key:0 for key in dates_given 
        
    }
dict_dates_2 = {
    key:[0,0,0] for key in dates_given
}
buses = [130, 2, 131, 132, 133, 134, 135, 136, 200, 137, 73, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 166, 167, 169]
total_buses = {}
total_dates = []
folder_img = "distance_vs_time"

def make_int(x):
    x=str(x)
    
    y=list(x)
    
    if "-" in y:
        y = x 
        year = int("".join(x[0:4]))
        month = int("".join(x[5:7]))
        day = int("".join(x[8:10]))
        return year , month , day
    else:
        year = int("".join(x[0:2]))
        month = int("".join(x[3:5]))
        day = int("".join(x[6:8]))
        return year , month , day

In [ ]:


    
total_lines = 0
def try_float_convert(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        
        return value
with open(save_path ,"w",newline="") as file:
    writer_file = csv.DictWriter(file , fieldnames=columns)
    writer_file.writeheader()
    for name in file_names:
        with open(name ,"r") as csv_file: 
            reader = csv.DictReader(csv_file)
            
            
            
            for line in reader:
                lines_to_write = [
                    try_float_convert(line[cols])  
                    for cols in columns 
                    #if line[cols] not in columns and line["alarm"]!="powerCut"
                    if line[cols] not in columns
                    ]
                try:
                    dict_temp = {
                    columns[0]:lines_to_write[0],
                    columns[1]:lines_to_write[1],
                    columns[2]:int(lines_to_write[2]),
                    columns[3]:lines_to_write[3],
                    columns[4]:lines_to_write[4],
                    columns[5]:lines_to_write[5],
                    columns[6]:lines_to_write[6],
                }
                    writer_file.writerow(dict_temp)
                    dict_temp={}
                    total_lines+=1 
                except:
                    pass 
    file.close()
print(f" a total of {total_lines} were created in {save_path}")
                

In [ ]:


def date_parser(dates):
    listed = []
    i=1
    print(dates[:10])
    for date in dates:
        
        i+=1        
        stinged = list(date)
        
        year = int(float("".join(stinged[0:4])))
        month = int(float("".join(stinged[5:7])))
        day=int(float("".join(stinged[8:10])))
        time1 = time(int(float("".join(stinged[11:13]))),int(float("".join(stinged[14:16]))) , int(float("".join(stinged[17:19]))))
        
        d1=datetime.date(year,month , day)
        
        listed.append(str(datetime.datetime.combine( d1, time1))) 
    return listed
df = pd.read_csv(save_path,header=0)

df["fixTime"] = date_parser(df["fixTime"])
df.to_csv(save_path , index = False)
print("done")

In [ ]:

            
with open(save_path , "r") as file:
    
    lines = csv.DictReader(file)
    temp_recorder = [[0,0,0] for _ in range(0 , 41)]

    

    for line in lines:
        #print(line)
        device_id = int(line["deviceId"])
        indexed = buses.index(device_id)
        
        time1 = line["fixTime"]
        time1 = time1.split() 
        yr , mon ,day = make_int(time1[0]) 
        hr,min,sec =  make_int(time1[1]) 
        # if time1[0] not in dates_given:
        #     dates_given.append(time1[0])
        
        
        
        
        
        #morning_peak 0 , free_hour 1 , evening_peak 2
        hour_time = 0 if 8<=hr<=11 else (1 if 11<hr<16 else (2 if 15<=hr<=19 else 1))
        temp_recorder[indexed][hour_time]+=1
        dict_dates[time1[0]]+=1
        some_data = dict_dates_2[time1[0]]
        some_data[hour_time]+=1 
        dict_dates_2[time1[0]] = some_data
    file.close()
with open(rcp , "w") as rcp_writer:
    writer = csv.DictWriter(rcp_writer , fieldnames=["device_id" , "morning_peak(8-11)" , "free_hour" , "evening_peak(3-7)"])
    writer.writeheader()
    for i in range(0,41):
        writer.writerow({
            "device_id":buses[i] , 
            "morning_peak(8-11)":temp_recorder[i][0],
            "free_hour":temp_recorder[i][1],
            "evening_peak(3-7)":temp_recorder[i][2]
            })      
    rcp_writer.close()
with open(dpd , "w") as dpd_writer:
    writer = csv.DictWriter(dpd_writer , fieldnames = ["date" ,"data_count" , "morning_rush" , "free" , "evening_rush"])
    writer.writeheader()
    for key,value in dict_dates.items():
        extra_val = dict_dates_2[key]
        temp_dict = {
            "date":key,
            "data_count":value,
            "morning_rush":extra_val[0],
            "free":extra_val[1],
            "evening_rush":extra_val[2]
        } 
        writer.writerow(temp_dict)
    dpd_writer.close()
            
    # for key,value in total_buses.iter():
    #     print(f"Bus {key} has {value} numbers of log in the dataset \n")

In [ ]:
# Extract categories and values
df = pd.read_csv(rcp)
categories = df['device_id']
group_a_values = df['morning_peak(8-11)']
group_b_values = df['free_hour']
group_c_values = df['evening_peak(3-7)']

# Set bar width and positions
bar_width = 0.2
index = np.arange(len(categories)) # Numerical positions for categories

# Create the figure and axes
fig, ax = plt.subplots(figsize=(20, 6))

# Plot each group of bars
bar1 = ax.bar(index - bar_width, group_a_values, bar_width, label='morning_peak(8-11)')
bar2 = ax.bar(index, group_b_values, bar_width, label='free_hour')
bar3 = ax.bar(index + bar_width, group_c_values, bar_width, label='evening_peak(3-7)')


ax.set_xlabel('Device_id')
ax.set_ylabel('No of datas')
ax.set_title('Mahanagar dataset 1.0')
ax.set_xticks(index)
ax.set_xticklabels(categories)
ax.legend()
plt.tight_layout() 
plt.show()

In [ ]:
# Extract categories and values
df = pd.read_csv(dpd)
categories = df['date']

group_a_values = df['morning_rush']
group_b_values = df['free']
group_c_values = df['evening_rush']
group_d_values = df["data_count"]
# Set bar width and positions
bar_width = 0.2
index = np.arange(len(categories)) # Numerical positions for categories

# Create the figure and axes
fig, ax = plt.subplots(figsize=(26, 6))

# Plot each group of bars
bar1 = ax.bar(index - bar_width, group_a_values, bar_width, label='morning_peak')
bar2 = ax.bar(index, group_b_values, bar_width, label='free_hour')
bar3 = ax.bar(index + bar_width, group_c_values, bar_width, label='evening_peak')
bar4 = ax.bar(index + bar_width*1.5, group_d_values, bar_width, label="total count") 

ax.set_xlabel('dates')
ax.set_ylabel("data_counts")
ax.set_title('Mahanagar dataset 1.1')
ax.set_xticks(index)
ax.set_xticklabels(categories)
ax.legend()
plt.tight_layout() 
plt.show()

In [ ]:

dict_info = {
    key:[] for key in buses
}
import math
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Returns the Haversine distance between two lat/lon points in meters.
    """
    R = 6371000  # Earth radius in meters

    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c
    
with open(save_path , "r") as file:
    
    take_date = dates_given[-1]
    exp_yr , exp_mon,exp_day = make_int(take_date)
    print(exp_yr)
    reader = csv.DictReader(file)
    for line in reader:
        
        date_choose = line["fixTime"]
        date_time = date_choose.split()
        
        yr , mon,day = make_int(date_time[0])
        curr_hr ,curr_min , curr_sec = 0,0,0
        if yr == exp_yr and exp_mon == mon and exp_day == day:
            hr,min,sec = make_int(date_time[1])
            if hr==curr_hr and min==curr_min and curr_sec == sec:
                
                continue
                
            #[(hr,min,sec),lat,lon]
            try:
                x = dict_info[int(line["deviceId"])][-1] 
                lat , lon = float(line["latitude"]) , float(line["longitude"])
            except:
                lat , lon = float(line["latitude"]) , float(line["longitude"])
            
                dict_info[int(line["deviceId"])].append([(hr,min,sec) , lat,lon])
                
                continue
            
            
            #calc. diffn in time 
            # time_taken = (abs(hr-prev_hr))*3600 + (abs(prev_min-min))*60 + (abs(sec-prev_sec))
            # dist = haversine_distance(lat,lon , prev_lat,prev_lon)
            dict_info[int(line["deviceId"])].append([(hr,min,sec) , lat,lon])
            curr_hr , curr_min,curr_sec = hr,min,sec
    file.close()     
    

In [ ]:
sorted_dict = {
    key:[[]] for key in buses
}
sorting_list = []
fixed_point = [27.678786911652914, 85.3494674406196] #koteshwor
def time_to_seconds(h, m, s):
    return h*3600 + m*60 + s


for key,value in dict_info.items():
    print(key)
    if value ==[]:
        continue
    sorted_values = sorted(value , key=lambda x:x[0] , reverse = False)
    prev_hr,prev_min ,prev_sec = 0,0,0
    prev_lat , prev_lon = 0,0
    total_distance_covered=0
    for x in sorted_values:
        hr ,min,sec=x[0]
        #print(prev_hr,prev_min , prev_sec , prev_lat,prev_lon)
        lat,lon = x[1],x[2]
        #print(key , hr,min,sec,lat,lon)
        
        if ((prev_hr ==0) and (prev_min==0) and (prev_sec==0)):
            prev_hr , prev_min,prev_sec = x[0]
            prev_lat , prev_lon = x[1],x[2]
            continue 
            #calc. diffn in time
        
        t1 = time_to_seconds(prev_hr, prev_min, prev_sec)
        t2 = time_to_seconds(hr, min, sec)

        time_taken = t2 - t1
        
        dist = haversine_distance(lat,lon , prev_lat,prev_lon)
        total_distance_covered +=dist

        fixed_dist = haversine_distance(lat,lon , fixed_point[0],fixed_point[1])
        sorting_list.append([(hr,min,sec) , lat,lon,time_taken , dist,total_distance_covered,fixed_dist])
        prev_hr , prev_min,prev_sec = x[0]
        prev_lat , prev_lon = x[1],x[2]
    #sorting_list = sorted(sorting_list,key = lambda x:x[0] , reverse = False)
    sorted_dict[key] = sorting_list 
    print(sorted_dict[key])
with open(save_path_2,"w") as file: 
    writer = csv.DictWriter(file , fieldnames=["deviceId","time","lat","lon","time_taken","distance","total_distance","from_koteshwor"])
    writer.writeheader()
    for key,value in sorted_dict.items():
        for val in value:
            if val ==[] or val[3]==0:
                continue
            
            dict_to_write = {
            "deviceId":key,
            "time":val[0],
            "lat":val[1],
            "lon":val[2],
            "time_taken":val[3],
            "distance":val[4],
            "total_distance":val[5],
            "from_koteshwor":val[6]
            }
            writer.writerow(dict_to_write)
    file.close()


In [ ]:
with open(save_path_2 , "r") as file:
    reader = csv.DictReader(file)
    #0 for time and 1 for distance covered
    dict_ploting = {
        key:[
            [],[]
            ] for key in buses
    }
    curr_dv_id =0 
    for line in reader:
        dict_ploting[int(line["deviceId"])][0].append(line["time"])
        dict_ploting[int(line["deviceId"])][1].append(float(line["total_distance"]))
for bus_id, (time_list, distance_list) in dict_ploting.items():
    plt.figure(figsize=(80, 40))
    plt.plot(time_list, distance_list, marker='o')
    plt.title(f'Distance vs Time for {bus_id}')
    plt.xlabel('Time')
    plt.ylabel('Distance Covered')
    plt.grid(True)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import csv
import ast   # <-- To safely convert "(15, 18, 29)" into a real tuple

dict_ploting = {key: [[], []] for key in buses}

with open(save_path_2, "r") as file:
    reader = csv.DictReader(file)
    for line in reader:
        dev_id = int(line["deviceId"])

        # Convert string "(15, 18, 29)" into real tuple
        t = ast.literal_eval(line["time"])
        dict_ploting[dev_id][0].append(t)

        # Total distance
        dist = float(line["total_distance"])
        dict_ploting[dev_id][1].append(dist)

# ---- PLOT ----
for bus_id, (time_list, distance_list) in dict_ploting.items():

    x = list(range(len(time_list)))   # equal interval x-axis

    plt.figure(figsize=(20, 8))
    plt.plot(x, distance_list, marker='o')

    plt.title(f"Distance vs Time for Bus {bus_id}")
    plt.xlabel("Equal Interval Points")
    plt.ylabel("Distance Covered")
    plt.grid(True)

    # Convert tuple to a readable label
    time_labels = [f"{h:02d}:{m:02d}:{s:02d}" for h, m, s in time_list]

    # Show max 20 x-tick labels to avoid clutter
    step = max(1, len(time_labels) // 20)
    plt.xticks(x[::step], time_labels[::step], rotation=45)

    plt.tight_layout()
    plt.show()
    plt.savefig(folder_img, dpi=300)


In [ ]:
bus_stations = { 
    "clockwise":{ 
        "koteshwor":[27.67329550, 85.34312386, 27.68230450, 85.35327614],
        "tinkune":[27.68159550, 85.34882386, 27.69060450, 85.35897614],
        "gausala":[27.70109550, 85.35232386, 27.71010450, 85.36247614],
        "chabahil":[27.72439550, 85.35102386, 27.73340450, 85.36117614],
        "narayan_gopal_chowk":[27.74699550, 85.33992386, 27.75600450, 85.35007614],
        "maharajgunj":[27.73319550, 85.32832386, 27.74220450, 85.33847614],
        "gangabu":[27.72959550, 85.31282386, 27.73860450, 85.32297614], # New Bus Park Area
        "balaju_bypass":[27.73029550, 85.29162386, 27.73930450, 85.30177614],
        "swoyambhu":[27.71179550, 85.27972386, 27.72080450, 85.28987614],
        "kalanki":[27.68549550, 85.26982386, 27.69450450, 85.27997614],
        "rabi_bhawan":[27.67999550, 85.28892386, 27.68900450, 85.29907614],
        "balkhu":[27.66949550, 85.29692386, 27.67850450, 85.30707614],
        "bagdol":[27.66349550, 85.31092386, 27.67250450, 85.32107614],
        "satdobato":[27.65349550, 85.31642386, 27.66250450, 85.32657614],
        "gwarko":[27.66529550, 85.32882386, 27.67430450, 85.33897614]
    }, 
    "anti-clockwise":{ 
        "koteshwor":[27.67329550, 85.34312386, 27.68230450, 85.35327614],
        "tinkune":[27.68159550, 85.34882386, 27.69060450, 85.35897614],
        "gausala":[27.70109550, 85.35232386, 27.71010450, 85.36247614],
        "chabahil":[27.72439550, 85.35102386, 27.73340450, 85.36117614],
        "narayan_gopal_chowk":[27.74699550, 85.33992386, 27.75600450, 85.35007614],
        "maharajgunj":[27.73319550, 85.32832386, 27.74220450, 85.33847614],
        "gangabu":[27.72959550, 85.31282386, 27.73860450, 85.32297614],
        "balaju_bypass":[27.73029550, 85.29162386, 27.73930450, 85.30177614],
        "swoyambhu":[27.71179550, 85.27972386, 27.72080450, 85.28987614],
        "kalanki":[27.68549550, 85.26982386, 27.69450450, 85.27997614],
        "rabi_bhawan":[27.67999550, 85.28892386, 27.68900450, 85.29907614],
        "balkhu":[27.66949550, 85.29692386, 27.67850450, 85.30707614],
        "bagdol":[27.66349550, 85.31092386, 27.67250450, 85.32107614],
        "satdobato":[27.65349550, 85.31642386, 27.66250450, 85.32657614],
        "gwarko":[27.66529550, 85.32882386, 27.67430450, 85.33897614]
    } 
}

In [ ]:
import pandas as pd
import numpy as np
import csv
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import defaultdict
import ast 
import sys 

# ===================================================================
# 0. SETUP: Helper Functions and Constants
# ===================================================================

# --- Constants & Paths ---
buses = [130, 2, 131, 132, 133, 134, 135, 136, 200, 137, 73, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 164, 165, 166, 167, 169]
save_path_2 = "data_tracking.csv"      # Original file (used as input source)
output_path_2 = "new_processed_data.csv" # NEW file for trajectory output (to avoid overwriting)
folder_img = "img_output" 
os.makedirs(folder_img, exist_ok=True)
fixed_point = [27.678786911652914, 85.3494674406196] # Koteshwor coordinates

# Bus Station Bounding Boxes
STATIONS = {
    "koteshwor":[27.67329550, 85.34312386, 27.68230450, 85.35327614],
    "tinkune":[27.68159550, 85.34882386, 27.69060450, 85.35897614],
    "gausala":[27.70109550, 85.35232386, 27.71010450, 85.36247614],
    "chabahil":[27.72439550, 85.35102386, 27.73340450, 85.36117614],
    "narayan_gopal_chowk":[27.74699550, 85.33992386, 27.75600450, 85.35007614],
    "maharajgunj":[27.73319550, 85.32832386, 27.74220450, 85.33847614],
    "gangabu":[27.72959550, 85.31282386, 27.73860450, 85.32297614],
    "balaju_bypass":[27.73029550, 85.29162386, 27.73930450, 85.30177614],
    "swoyambhu":[27.71179550, 85.27972386, 27.72080450, 85.28987614],
    "kalanki":[27.68549550, 85.26982386, 27.69450450, 85.27997614],
    "rabi_bhawan":[27.67999550, 85.28892386, 27.68900450, 85.29907614],
    "balkhu":[27.66949550, 85.29692386, 27.67850450, 85.30707614],
    "bagdol":[27.66349550, 85.31092386, 27.67250450, 85.32107614],
    "satdobato":[27.65349550, 85.31642386, 27.66250450, 85.32657614],
    "gwarko":[27.66529550, 85.32882386, 27.67430450, 85.33897614]
}
STATION_NAMES = list(STATIONS.keys())

# --- Helper Functions ---
def time_to_seconds(h, m, s):
    return h * 3600 + m * 60 + s

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # Earth radius in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def get_current_station(lat, lon):
    for name, bbox in STATIONS.items():
        min_lat, min_lon, max_lat, max_lon = bbox
        if min_lat <= lat <= max_lat and min_lon <= lon <= max_lon:
            return name
    return None

# ===================================================================
# 1. DATA LOADING LOGIC (Loads from data_tracking.csv)
# ===================================================================

dict_info = {key: [] for key in buses}
data_loaded_from_csv = False

if os.path.exists(save_path_2):
    print(f"Attempting to load raw data from original file: {save_path_2}...")
    try:
        df_load = pd.read_csv(save_path_2)
        df_load = df_load[df_load['time'].astype(str).str.startswith('(')]
        
        for index, row in df_load.iterrows():
            deviceId = row['deviceId']
            time_tuple = ast.literal_eval(row['time'])
            
            # Reconstruct the raw data format: [[h, m, s], lat, lon]
            dict_info.setdefault(deviceId, []).append([
                [time_tuple[0], time_tuple[1], time_tuple[2]], 
                row['lat'], 
                row['lon']
            ])
        
        if any(dict_info.values()):
            data_loaded_from_csv = True
            print(f"Successfully loaded raw data for {len([k for k, v in dict_info.items() if v])} buses.")
            
    except Exception as e:
        print(f"Error loading {save_path_2}: {e}. Falling back to placeholder data.", file=sys.stderr)
        
# FALLBACK: Placeholder data for bus 130 if no real data was loaded
if not data_loaded_from_csv:
    print("WARNING: Input data file was not found/empty. Using placeholder data for bus 130.")
    dict_info[130] = [
        [[7, 0, 0], 27.6780, 85.3480],      
        [[7, 2, 0], 27.6850, 85.3000],      
        [[7, 10, 0], 27.6900, 85.2750],     
        [[7, 13, 0], 27.6900, 85.2750],     
        [[7, 14, 0], 27.6950, 85.2700],     
        [[7, 30, 0], 27.7300, 85.3180],     
        [[7, 35, 0], 27.7300, 85.3180],     
        [[8, 0, 0], 27.6780, 85.3480]       
    ]

buses_with_data = [k for k, v in dict_info.items() if v]

# ===================================================================
# 2. DATA PROCESSING & SAVING TO NEW CSV
# ===================================================================

sorted_dict = {key: [] for key in buses}

for key, value in dict_info.items():
    if key not in buses_with_data: continue
    
    print(f"\nCalculating trajectory metrics for bus {key}...")
    
    sorted_values = sorted(value, key=lambda x: time_to_seconds(*x[0]), reverse=False)
    
    prev_hr, prev_min, prev_sec = 0, 0, 0
    prev_lat, prev_lon = 0, 0
    total_distance_covered = 0
    
    sorting_list = []
    
    for x in sorted_values:
        hr, min, sec = x[0]
        lat, lon = x[1], x[2]
        
        if ((prev_hr == 0) and (prev_min == 0) and (prev_sec == 0)):
            prev_hr, prev_min, prev_sec = x[0]
            prev_lat, prev_lon = x[1], x[2]
            continue
            
        t1 = time_to_seconds(prev_hr, prev_min, prev_sec)
        t2 = time_to_seconds(hr, min, sec)

        time_taken = t2 - t1
        
        dist = haversine_distance(lat, lon, prev_lat, prev_lon)
        total_distance_covered += dist

        fixed_dist = haversine_distance(lat, lon, fixed_point[0], fixed_point[1])
        sorting_list.append([(hr, min, sec), lat, lon, time_taken, dist, total_distance_covered, fixed_dist])
        
        prev_hr, prev_min, prev_sec = x[0]
        prev_lat, prev_lon = x[1], x[2]
        
    sorted_dict[key] = sorting_list
    
# Write the processed data to the NEW file (output_path_2)
print(f"\nSaving detailed trajectory to NEW file: {output_path_2}...")
with open(output_path_2, "w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["deviceId", "time", "lat", "lon", "time_taken", "distance", "total_distance", "from_koteshwor"])
    writer.writeheader()
    for key, value in sorted_dict.items():
        for val in value:
            if val == [] or val[3] == 0:
                continue
            
            dict_to_write = {
                "deviceId": key,
                "time": str(val[0]), 
                "lat": val[1],
                "lon": val[2],
                "time_taken": val[3],
                "distance": val[4],
                "total_distance": val[5],
                "from_koteshwor": val[6]
            }
            writer.writerow(dict_to_write)
    print(f"{output_path_2} saved. Original file {save_path_2} was not modified.")
    file.close()

# ===================================================================
# 3. ADVANCED ANALYSIS: Rounds, Waiting, and Travel Time
# ===================================================================

print("\nStarting Advanced Analysis (Rounds, Waiting, Travel Time)...")

# Load data from the NEWLY processed file
df_full = pd.read_csv(output_path_2)
df_full['time'] = df_full['time'].apply(ast.literal_eval)
df_full['total_seconds'] = df_full['time'].apply(lambda t: time_to_seconds(t[0], t[1], t[2]))
df_full['station'] = df_full.apply(lambda row: get_current_station(row['lat'], row['lon']), axis=1)

round_trip_data = defaultdict(list)
all_wait_times = defaultdict(list)
all_travel_times = defaultdict(list) 

total_passes = 0

for deviceId in buses_with_data:
    df = df_full[df_full['deviceId'] == deviceId].sort_values('total_seconds').reset_index(drop=True)
    
    current_station = None
    station_entry_time = None
    koteshwor_exit_time = None
    current_lap = 1
    last_station_name = None
    
    for i, row in df.iterrows():
        next_station = row['station']
        
        # --- Station Entry/Exit and Waiting Time/Travel Time ---
        if next_station and next_station != current_station:
            station_entry_time = row['total_seconds']
            
            if last_station_name == 'koteshwor' and next_station != 'koteshwor' and koteshwor_exit_time is not None:
                travel_time = station_entry_time - koteshwor_exit_time
                all_travel_times[next_station].append(travel_time)

        elif not next_station and current_station:
            if station_entry_time is not None:
                wait_time = row['total_seconds'] - station_entry_time
                all_wait_times[current_station].append(wait_time)
                station_entry_time = None
            
            last_station_name = current_station

        # --- Round Trip Detection ---
        if current_station == 'koteshwor' and next_station != 'koteshwor':
            koteshwor_exit_time = row['total_seconds']
            
        elif current_station != 'koteshwor' and next_station == 'koteshwor':
            if koteshwor_exit_time is not None:
                lap_duration = row['total_seconds'] - koteshwor_exit_time
                round_trip_data[deviceId].append({
                    'lap_number': current_lap,
                    'start_time_seconds': koteshwor_exit_time,
                    'end_time_seconds': row['total_seconds'],
                    'time_taken_seconds': lap_duration
                })
                current_lap += 1
                total_passes += 1
                koteshwor_exit_time = None 
                
        current_station = next_station
        
print(f"Total calculated passes across all buses: {total_passes}")

# ===================================================================
# 4. GENERATE CSV FOR ROUND TRIPS (Per-Bus CSV)
# ===================================================================

print("\nGenerating round trip CSV files...")
for deviceId, laps in round_trip_data.items():
    save_path_bus = f"{deviceId}_round_trips.csv"
    with open(save_path_bus, "w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["busId", "lap_number", "start_time_seconds", "end_time_seconds", "time_taken_seconds"])
        writer.writeheader()
        for lap in laps:
            writer.writerow(lap)
    print(f"Generated {save_path_bus}")

# ===================================================================
# 5. HEATMAP DATA CALCULATION 
# ===================================================================

print("\nCalculating Heatmap metrics...")

heatmap_data = pd.DataFrame(index=STATION_NAMES, columns=STATION_NAMES, dtype=float)

# 1. Diagonal: Average Waiting Time (Station X vs. Station X)
for station in STATION_NAMES:
    if all_wait_times[station]:
        avg_wait_time = np.mean(all_wait_times[station])
        heatmap_data.loc[station, station] = avg_wait_time
    else:
        heatmap_data.loc[station, station] = 0.0

# 2. Off-Diagonal: Average Travel Time (Koteshwor vs. Station X)
for station in STATION_NAMES:
    if station == 'koteshwor':
        continue
        
    if all_travel_times[station]:
        avg_travel_time = np.mean(all_travel_times[station])
        heatmap_data.loc['koteshwor', station] = avg_travel_time
        
heatmap_data = heatmap_data.fillna(0.0)
print("\n--- Heatmap Data (Time in Seconds) ---")
print(heatmap_data)

# ===================================================================
# 6. GENERATE AND SAVE HEATMAP
# ===================================================================

print(f"\nGenerating and saving heatmap to {folder_img}/bus_time_heatmap.png...")
plt.figure(figsize=(15, 12))
sns.heatmap(
    heatmap_data, 
    annot=True, 
    fmt=".1f", 
    cmap="YlGnBu", 
    linewidths=.5,
    cbar_kws={'label': 'Time (Seconds)'}
)
plt.title('Bus Time Metrics: Waiting Time (Diagonal) & Koteshwor Travel Time (Row)')
plt.xlabel('Destination Station')
plt.ylabel('Source Station')
plt.tight_layout()
plt.savefig(f'{folder_img}/bus_time_heatmap.png')
print("Heatmap saved successfully.")